# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [241]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [242]:
csv_data_file = "../output_data/cities.csv"
cities_df=pd.read_csv(csv_data_file)
cities_df=cities_df.dropna()
cities_df.head()


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
0,Roebourne,-20.78,117.13,81.46,66,47,2.01,AU,12/15/20 18:47
1,Port Alfred,-33.59,26.89,64.99,90,2,1.99,ZA,12/15/20 18:47
2,Yumen,40.28,97.20,9.66,87,71,1.59,CN,12/15/20 18:47
3,Puerto Ayora,-0.74,-90.35,73.00,89,83,5.01,EC,12/15/20 18:47
4,Punta Arenas,-53.15,-70.92,50.00,61,0,32.21,CL,12/15/20 18:47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [243]:
gmaps.configure(api_key=g_key)
locations=cities_df[['Lat', 'Lng']].astype(float)
humidity=cities_df['Humidity'].astype(str).tolist()

In [244]:
# Plot Heatmap
#fig = gmaps.figure()
fig = gmaps.figure(center=(30, -20), zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,                                 
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [245]:
#Narrow down the cities with temperature <70 and >40 and humidity = 0
reduced_df= cities_df.loc[(cities_df['Humidity'] > 90 ) & (cities_df['Max temp'] > 55 ) & (cities_df['Max temp'] <72 )]
reduced_df.dropna()


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
88,Chiang Klang,19.29,100.86,62.60,100,25,3.36,TH,12/15/20 18:47
93,Biankouma,7.74,-7.61,70.32,93,94,2.24,CI,12/15/20 18:47
176,Cidreira,-30.18,-50.21,71.56,92,3,10.67,BR,12/15/20 18:47
186,Airai,-8.93,125.41,68.67,93,94,3.15,TL,12/15/20 18:47
212,Tacoronte,28.48,-16.41,63.00,93,90,13.87,ES,12/15/20 18:47
217,San Cristobal,7.77,-72.22,66.42,91,100,0.67,VE,12/15/20 18:47
225,Lashio,22.93,97.75,56.75,100,16,0.60,MM,12/15/20 18:47
226,Kinkala,-4.36,14.76,70.45,93,100,2.01,CG,12/15/20 18:47
306,Castro,-24.79,-50.01,64.44,94,1,3.13,BR,12/15/20 18:47
428,Codrington,-38.27,141.97,56.28,93,100,10.27,AU,12/15/20 18:47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [246]:
#Below gives key error 
hotel_df=reduced_df.loc[:,["City", "Lat", "Lng", "Country"]]
hotel_df['Hotel Name']=np.nan
hotel_df

,City,Lat,Lng,Country,Hotel Name
88,Chiang Klang,19.29,100.86,TH,NaN
93,Biankouma,7.74,-7.61,CI,NaN
176,Cidreira,-30.18,-50.21,BR,NaN
186,Airai,-8.93,125.41,TL,NaN
212,Tacoronte,28.48,-16.41,ES,NaN
217,San Cristobal,7.77,-72.22,VE,NaN
225,Lashio,22.93,97.75,MM,NaN
226,Kinkala,-4.36,14.76,CG,NaN
306,Castro,-24.79,-50.01,BR,NaN
428,Codrington,-38.27,141.97,AU,NaN


In [247]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_type = "hotel"
#target_radius = 5000
params = {
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
        }


In [251]:
#Construct Lat and Lng for each line in hotel_df
for index, line in hotel_df.iterrows():
    params["location"]= f"{line['Lat']}, {line['Lng']}"
    city_name=line['City']
    #Request the API
    print(f'Requesting info for the city {city_name}')
    # run a request using the params dictionary
    response = requests.get(base_url, params=params).json()
 #   print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        place_name = response["results"][0]['name']
        print(f'Hotel name : {place_name}')
        hotel_df.loc[index, 'Hotel Name']= place_name
    except (KeyError, IndexError):
        print(f'Missing hotels for City {city_name}')


Requesting info for the city Chiang Klang
Hotel name : Sangthongresort
Requesting info for the city Biankouma
Hotel name : Hôtel Licolais
Requesting info for the city Cidreira
Hotel name : Hotel Castelo
Requesting info for the city Airai
Missing hotels for City Airai
Requesting info for the city Tacoronte
Hotel name : OYO San Diego
Requesting info for the city San Cristobal
Hotel name : Lidotel Hotel Boutique San Cristobal
Requesting info for the city Lashio
Hotel name : Two Elephants Hotel
Requesting info for the city Kinkala
Hotel name : Hotel la Vernande
Requesting info for the city Castro
Hotel name : Central Palace Hotel - Castro (PR)
Requesting info for the city Codrington
Hotel name : Codrington Gardens Bed And Breakfast
Requesting info for the city Kambove
Missing hotels for City Kambove


In [252]:

hotel_df

,City,Lat,Lng,Country,Hotel Name
88,Chiang Klang,19.29,100.86,TH,Sangthongresort
93,Biankouma,7.74,-7.61,CI,Hôtel Licolais
176,Cidreira,-30.18,-50.21,BR,Hotel Castelo
186,Airai,-8.93,125.41,TL,NaN
212,Tacoronte,28.48,-16.41,ES,OYO San Diego
217,San Cristobal,7.77,-72.22,VE,Lidotel Hotel Boutique San Cristobal
225,Lashio,22.93,97.75,MM,Two Elephants Hotel
226,Kinkala,-4.36,14.76,CG,Hotel la Vernande
306,Castro,-24.79,-50.01,BR,Central Palace Hotel - Castro (PR)
428,Codrington,-38.27,141.97,AU,Codrington Gardens Bed And Breakfast


In [254]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [256]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))